In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/data1.csv', header=None)
labels = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/label1.csv', header=None)

In [3]:
labels.head()

,0
0,3
1,2
2,2
3,3
4,3


In [4]:
labels.shape, dataset.shape

((20000, 1), (20000, 20))

In [5]:
labels[:5]

,0
0,3
1,2
2,2
3,3
4,3


In [6]:
dataset = dataset.values
labels = labels.values

In [7]:
print(dataset.shape, labels.shape)

(20000, 20) (20000, 1)


In [8]:
print(np.min(labels), np.max(labels))
print(np.unique(labels))

0 4
[0 1 2 3 4]


In [9]:
max_classes = np.max(labels) + 1

In [10]:
max_classes

5

In [11]:
onehot_label = np.eye(max_classes)[labels]

In [12]:
onehot_label.shape

(20000, 1, 5)

In [13]:
onehot_label[:5]

array([[[0., 0., 0., 1., 0.]],

       [[0., 0., 1., 0., 0.]],

       [[0., 0., 1., 0., 0.]],

       [[0., 0., 0., 1., 0.]],

       [[0., 0., 0., 1., 0.]]])

In [14]:
onehot_label = onehot_label.reshape(20000, 5)
print(onehot_label.shape)

(20000, 5)


In [15]:
train_dataset, test_dataset = train_test_split(dataset, train_size=0.8, test_size=0.2)
train_label, test_label = train_test_split(onehot_label, train_size=0.8, test_size=0.2)

In [16]:
hidden_neurons = 15

In [17]:
w1=np.random.normal(0, 1/np.sqrt(20), (20,hidden_neurons))
w2=np.random.normal(0, 1/np.sqrt(hidden_neurons), (hidden_neurons,5))

In [18]:
print(w1, w2)

[[-3.46426354e-01  6.20164970e-01 -1.69362154e-01 -2.46164724e-01
   1.38990596e-01  3.61594968e-01 -1.00423479e-01 -3.46403907e-01
   7.56099651e-02 -5.25009724e-02 -1.68578816e-01  2.85102648e-01
  -1.39205925e-01  3.86815799e-01 -6.61687725e-02]
 [-3.75248861e-01 -1.85504106e-01 -2.59315919e-01  2.41559877e-01
  -2.45999679e-01  9.32308818e-02 -2.15258997e-01  6.45194074e-02
   1.72031275e-01  4.79264155e-02 -2.80251162e-02  8.12330297e-02
  -3.18015670e-02 -2.15876639e-01  1.96347540e-02]
 [-6.43688549e-02  5.39602150e-01 -2.63346341e-01  4.72005381e-01
   6.02202758e-02  1.46055035e-01 -8.47886842e-02 -1.25771980e-01
  -3.24491248e-01 -2.11145359e-01 -1.66785113e-01  3.52913942e-01
  -4.89313970e-02  2.37076308e-01  1.48327920e-01]
 [ 1.08595693e-01 -1.02715726e-01 -2.14856340e-01 -1.34488721e-01
   2.26155448e-01 -2.32604883e-01 -3.92008872e-02  2.48538612e-01
  -6.13420226e-02  8.23547860e-02 -7.13773892e-03  9.20955498e-02
  -1.39532448e-01 -3.75512535e-01 -1.75061455e-01]
 [-1

In [19]:
b1 = np.zeros(hidden_neurons)
b2 = np.zeros(5)

In [20]:
def activatetion_function(X, choice):
    if (choice == 1):
        return np.tanh(X)
    if (choice == 2):
        numerator = np.exp(X)
        return numerator/np.sum(numerator, axis=1, keepdims=True)

In [21]:
learning_rate = 0.003

In [22]:
batch = 1000

In [23]:
def cost_function(y, t):
    return -(t*(np.log(y)) + (1 - t)*np.log(1-y))

In [24]:
def forward_propogation(data):
    a0 = data
    z1 = np.dot(a0, w1) + b1
    a1 = activatetion_function(z1, 1)
    z2 = np.dot(a1, w2) + b2
    a2 = activatetion_function(z2, 2)
    return a0, z1, a1, z2, a2

In [25]:
def accuracy(y, t, size):
    pred_class = np.argmax(t, axis=1)
    label_class = np.argmax(y, axis=1)
    compare_label_pred = np.equal(pred_class, label_class)
    numerator = np.sum(compare_label_pred)
    return (numerator/size)*100

In [26]:
epochs = 100

In [27]:
for epoch in range(epochs):
    for iteration in range(int(len(train_dataset)/batch)):
        a0, z1, a1, z2, a2 = forward_propogation(train_dataset[iteration*batch:(iteration+1)*batch, :])
        y = a2
        label_batch = train_label[iteration*batch:(iteration+1)*batch, :]
        
        del2=(y-label_batch)
        del1=np.dot(del2,w2.T)*(a1-pow(a1,2))
        dcdw2=np.dot(a1.T,del2)
        dcdw1=np.dot(a0.T,del1)
        dcdb1=np.sum(del1,axis=0)
        dcdb2=np.sum(del2,axis=0)
        
        w1 = w1 - learning_rate*dcdw1
        w2 = w2 - learning_rate*dcdw2
        b1 = b1 - learning_rate*dcdb1
        b2 = b2 - learning_rate*dcdb2
        
    a0, z1, a1, z2, a2 = forward_propogation(train_dataset)
#     if epoch>9800:
    print('Training Cost: ', np.sum(cost_function(a2, train_label))/np.shape(train_dataset)[0])
    print('Train Accuracy:', accuracy(train_label, a2, len(train_label)))

Training Cost:  2.5158754729312354
Train Accuracy: 20.63125
Training Cost:  3.932155024862735
Train Accuracy: 19.9625
Training Cost:  5.4059826436748315
Train Accuracy: 20.075000000000003
Training Cost:  12.216400900941059
Train Accuracy: 20.13125
Training Cost:  18.518190516534553
Train Accuracy: 19.60625
Training Cost:  14.731177789484603
Train Accuracy: 19.693749999999998
Training Cost:  10.80225300463552
Train Accuracy: 20.150000000000002
Training Cost:  6.478990164278091
Train Accuracy: 20.11875
Training Cost:  12.192339839284369
Train Accuracy: 19.875
Training Cost:  12.465398363397691
Train Accuracy: 19.71875
Training Cost:  12.045928156907896
Train Accuracy: 20.2125
Training Cost:  7.061592623757859
Train Accuracy: 20.150000000000002
Training Cost:  7.562442534474412
Train Accuracy: 20.025000000000002
Training Cost:  15.192156744126367
Train Accuracy: 19.66875
Training Cost:  16.73138155633933
Train Accuracy: 19.787499999999998
Training Cost:  12.866074685864426
Train Accuracy:

In [28]:
# np.sum?